#### **Dans ce notebook, nous allons charger, nettoyer et préparer les données extraites d'Avito. Le résultat final sera enregistré dans un fichier CSV propre prêt pour l'analyse.**

In [1]:
# Les importations nécessaires
import pandas as pd
import numpy as np
import os

On charge le fichier brut


In [2]:
# Chargement du fichier source (sans le modifier)
df = pd.read_csv(
    '../data/raw/avito_vendre.csv',
    engine='python',
    on_bad_lines='skip',
    na_values=['null']
)

print("Taille initiale :", len(df))
df.head()

Taille initiale : 6053


,id,ville,prix,surface,quartier,type_bien,nb_chambres,nb_salle_de_bains,url_annonce,date_annonce
0,57346043.0,Casablanca,16674 DH,287 m2,Hay Laymouna,Maison,7,0,https://www.avito.ma/fr/hay_laymouna/maisons/M...,il y a 22 heures
1,57029300.0,Tanger,25846 DH,113 m2,Ahlane,Maison,12,0,https://www.avito.ma/fr/ahlane/maisons/Maison_...,il y a 2 heures
2,57374711.0,Marrakech,19454 DH,260 m2,Route de Tahanaoute,Villa,4,0,https://www.avito.ma/fr/route_de_tahanaoute/ma...,il y a 7 heures
3,57333583.0,Casablanca,255682 DH,102 m2,Sidi Maarouf,Maison,12,0,https://www.avito.ma/fr/sidi_maarouf/maisons/M...,il y a 19 minutes
4,56921942.0,Casablanca,19454 DH,378 m2,Sidi Bernoussi,Maison,4,0,https://www.avito.ma/fr/sidi_bernoussi/maisons...,il y a 55 minutes


## Nettoyage de base


Dans cette phase, nous supprimons les colonnes inutiles, les doublons et les lignes où les informations cruciales (comme la ville) sont manquantes.

In [3]:
# 1. Supprimer la colonne date_annonce
if 'date_annonce' in df.columns:
    df.drop(columns=['date_annonce'], inplace=True)

# 2. Supprimer les doublons
df.drop_duplicates(inplace=True)

# 3. Supprimer les lignes où ville est null
df.dropna(subset=['ville'], inplace=True)

# 4. Supprimer les lignes où nb_chambres est 0 (ou vide)
df = df[df['nb_chambres'] > 0]

print("Taille après nettoyage de base :", len(df))

Taille après nettoyage de base : 5232


### Prétraitement : Suppression des unités (DH, m2) et conversion en Float


In [4]:
def clean_numeric_col(series, unit):
    return series.astype(str).str.replace(unit, '', case=False)\
                 .str.replace(r'\s+', '', regex=True)\
                 .replace(['nan', 'None', ''], '0')\
                 .astype(float)

# Nettoyer prix (supprimer DH)
df['prix'] = clean_numeric_col(df['prix'], 'DH')

# Nettoyer surface (supprimer m2)
df['surface'] = clean_numeric_col(df['surface'], 'm2')

print("Types après conversion :")
print(df[['prix', 'surface']].dtypes)

Types après conversion :
prix       float64
surface    float64
dtype: object


### Test de la logique d'imputation (Type de bien)

In [5]:
# 1. Trouver l'index de la première ligne où type_bien est null
try:
    idx_test = df[df['type_bien'].isna()].index[0]
    
    print(f"--- AVANT (Index: {idx_test}) ---")
    display(df.loc[[idx_test]])

    # 2. Appliquer la logique de nettoyage et d'imputation
    df['type_bien'] = df['type_bien'].str.strip().str.capitalize()
    
    def impute_type(row):
        if pd.isna(row['type_bien']) or row['type_bien'] == 'None':
            return 'Maison' if row['nb_chambres'] <= 6 else 'Villa'
        return row['type_bien']

    df['type_bien'] = df.apply(impute_type, axis=1)

    print(f"--- APRÈS (Index: {idx_test}) ---")
    display(df.loc[[idx_test]])

except IndexError:
    print("Aucune ligne avec type_bien = null n'a été trouvée dans votre dataset.")

--- AVANT (Index: 9) ---


,id,ville,prix,surface,quartier,type_bien,nb_chambres,nb_salle_de_bains,url_annonce
9,57415933.0,Casablanca,16674.0,140.0,Hay Sadri,NaN,3,0,https://www.avito.ma/fr/hay_sadri/maisons/Imme...


--- APRÈS (Index: 9) ---


,id,ville,prix,surface,quartier,type_bien,nb_chambres,nb_salle_de_bains,url_annonce
9,57415933.0,Casablanca,16674.0,140.0,Hay Sadri,Maison,3,0,https://www.avito.ma/fr/hay_sadri/maisons/Imme...


### Imputation du nombre de salles de bains
**Règle appliquée :**
- Si la valeur est `0` ou `Null` :
    - Si `nb_chambres` ≤ 3  ➔  **1** salle de bain.
    - Sinon  ➔  **(nb_chambres / 2) - 1**.

In [6]:
def fix_bathrooms(row):
    # Si c'est 0 ou NaN
    if row['nb_salle_de_bains'] == 0 or pd.isna(row['nb_salle_de_bains']):
        if row['nb_chambres'] <= 3:
            return 1
        else:
            return (row['nb_chambres'] / 2) - 1
    return row['nb_salle_de_bains']

df['nb_salle_de_bains'] = df.apply(fix_bathrooms, axis=1)

### Traitement des Prix et Surfaces à 0
**Stratégie d'imputation :**
1. **Villes cibles** (Casablanca, Rabat, Tanger, Marrakech) : Remplacement des `0` par la moyenne groupée par *Ville* et *Type de bien*.
2. **Fallback global** : Pour les surfaces restantes à `0`, application de la moyenne générale par *Type de bien*.

In [7]:
target_cities = ['Casablanca', 'Rabat', 'Tanger', 'Marrakech']

for col in ['prix', 'surface']:
    # 1. On calcule la table de référence des moyennes par Ville et par Type
    # On ne prend que les valeurs strictement supérieures à 0
    df_valid = df[df[col] > 0]
    lookup_villes = df_valid.groupby(['ville', 'type_bien'])[col].mean()
    
    # 2. On calcule aussi une moyenne globale par Type de bien (au cas où une ville n'a que des 0)
    lookup_global = df_valid.groupby('type_bien')[col].mean()

    # 3. Création du masque pour les lignes à 0 dans les villes cibles
    mask = (df[col] == 0) & (df['ville'].isin(target_cities))

    # 4. Remplacement intelligent
    # On essaie d'abord la moyenne Ville + Type
    df.loc[mask, col] = df.loc[mask].apply(
        lambda row: lookup_villes.get((row['ville'], row['type_bien']), 
                    lookup_global.get(row['type_bien'], 0)), 
        axis=1
    )

    # 5. Sécurité finale : s'il reste des 0 ou des NaN (pour les villes hors liste)
    df[col] = df[col].replace(0, np.nan) # On transforme les 0 en NaN pour utiliser fillna
    df[col] = df[col].fillna(df.groupby('type_bien')[col].transform('mean'))
    
    # Si vraiment il n'y a aucune donnée pour un type de bien (ex: seul terrain du site)
    df[col] = df[col].fillna(0)

print("Nettoyage des prix et surfaces terminé sans NaN.")

Nettoyage des prix et surfaces terminé sans NaN.


### Ajustement final des prix (Villa/Maison) et typage strict des données

In [8]:
# 1. Multiplier prix des villas et maisons par 179,92
mask_v_m = df['type_bien'].isin(['Villa', 'Maison'])

df.loc[mask_v_m, 'prix'] = (
    df.loc[mask_v_m, 'prix'] * 179.92
).round(1).astype(float)


# 2. Nettoyage spécifique de l'ID pour éviter le ".0"
# On le convertit en float puis int pour enlever la virgule, puis en string
df['id'] = pd.to_numeric(df['id'], errors='coerce').fillna(0).astype(int).astype("string")

# 3. Conversion finale des types
# Notez l'utilisation de "string" pour les colonnes de texte
df = df.astype({
    'ville': 'string',
    'quartier': 'string',
    'type_bien': 'string',
    'url_annonce': 'string',
    'nb_chambres': 'int',
    'nb_salle_de_bains': 'int',
    'prix': 'float',
    'surface': 'float'
})

print("Vérification des types :")
print(df.dtypes)
print("\nRépartition des types de biens :")
print(df['type_bien'].value_counts())
df.head()

Vérification des types :
id                   string[python]
ville                string[python]
prix                        float64
surface                     float64
quartier             string[python]
type_bien            string[python]
nb_chambres                   int64
nb_salle_de_bains             int64
url_annonce          string[python]
dtype: object

Répartition des types de biens :
type_bien
Villa          3260
Maison         1883
Appartement      86
Terrain           3
Name: count, dtype: Int64


,id,ville,prix,surface,quartier,type_bien,nb_chambres,nb_salle_de_bains,url_annonce
0,57346043,Casablanca,2999986.1,287.0,Hay Laymouna,Maison,7,2,https://www.avito.ma/fr/hay_laymouna/maisons/M...
1,57029300,Tanger,4650212.3,113.0,Ahlane,Maison,12,5,https://www.avito.ma/fr/ahlane/maisons/Maison_...
2,57374711,Marrakech,3500163.7,260.0,Route de Tahanaoute,Villa,4,1,https://www.avito.ma/fr/route_de_tahanaoute/ma...
3,57333583,Casablanca,46002305.4,102.0,Sidi Maarouf,Maison,12,5,https://www.avito.ma/fr/sidi_maarouf/maisons/M...
4,56921942,Casablanca,3500163.7,378.0,Sidi Bernoussi,Maison,4,1,https://www.avito.ma/fr/sidi_bernoussi/maisons...


### Nettoyage, Filtrage et Imputation des types de biens

In [9]:
# 1. Nettoyage des espaces blancs et uniformisation de la casse (Première lettre en Majuscule)
df['type_bien'] = df['type_bien'].str.strip().str.capitalize()
df['ville'] = df['ville'].str.strip().str.capitalize()

# 2. Supprimer les types indésirables avec certitude
excluded_types = ['Bureau', 'Magasin', 'Terrain',"Commerce"]
df = df[~df['type_bien'].isin(excluded_types)]

# 3. Imputer type_bien si null (règle nbr_chambres)
def impute_type(row):
    if pd.isna(row['type_bien']) or row['type_bien'] == 'None':
        return 'Maison' if row['nb_chambres'] <= 6 else 'Villa'
    return row['type_bien']

df['type_bien'] = df.apply(impute_type, axis=1)

print("Types de biens restants :", df['type_bien'].unique())

Types de biens restants : ['Maison' 'Villa' 'Appartement']


In [10]:
df = df.rename(columns={"nb_salle_de_bains": "nb_salle_de_bain"})

### Contrôle Qualité 

In [11]:
print("===  AUDIT DE QUALITÉ FINAL (POST-TYPAGE) ===")

# 1. Tableau de bord des types et valeurs manquantes
audit = pd.DataFrame({
    'Type': df.dtypes,
    'Manquants (NaN)': df.isna().sum(),
    'Exemple': df.iloc[0] if len(df) > 0 else "N/A"
})
display(audit)

# 2. Vérifications spécifiques
print("-" * 50)

# Vérification de l'ID (ne doit pas contenir de ".0")
id_pb = df['id'].str.contains(r'\.0$').sum()
print(f"✅ ID propres (pas de .0) : {'OUI' if id_pb == 0 else f'NON ({id_pb} erreurs)'}")

# Vérification des types String (doit être 'string' et non 'object')
obj_cols = [col for col in df.columns if df[col].dtype == 'object']
print(f"✅ Conversion String : {'OUI' if not obj_cols else f'NON (colonnes {obj_cols} encore en object)'}")

# Vérification de la suppression des types exclus (Bureau, Commerce, Terrain)
types_exclus = ['Bureau', 'Commerce', 'Terrain', 'Magasin']
restants = df[df['type_bien'].isin(types_exclus)]
print(f"✅ Types exclus supprimés : {'OUI' if len(restants) == 0 else f'NON ({len(restants)} restants : {restants.type_bien.unique()})'}")


print("-" * 50)
print(f"📊 Lignes totales : {len(df)}")
if len(restants) > 0:
    print("⚠️ ALERTE : Il reste des types de biens à supprimer avant la sauvegarde !")
else:
    print(" TOUT EST PRÊT : La data est propre et bien typée.")

df.head()

===  AUDIT DE QUALITÉ FINAL (POST-TYPAGE) ===


,Type,Manquants (NaN),Exemple
id,string[python],0,57346043
ville,string[python],0,Casablanca
prix,float64,0,2999986.1
surface,float64,0,287.0
quartier,string[python],0,Hay Laymouna
type_bien,object,0,Maison
nb_chambres,int64,0,7
nb_salle_de_bain,int64,0,2
url_annonce,string[python],0,https://www.avito.ma/fr/hay_laymouna/maisons/M...


--------------------------------------------------
✅ ID propres (pas de .0) : OUI
✅ Conversion String : NON (colonnes ['type_bien'] encore en object)
✅ Types exclus supprimés : OUI
--------------------------------------------------
📊 Lignes totales : 5229
 TOUT EST PRÊT : La data est propre et bien typée.


,id,ville,prix,surface,quartier,type_bien,nb_chambres,nb_salle_de_bain,url_annonce
0,57346043,Casablanca,2999986.1,287.0,Hay Laymouna,Maison,7,2,https://www.avito.ma/fr/hay_laymouna/maisons/M...
1,57029300,Tanger,4650212.3,113.0,Ahlane,Maison,12,5,https://www.avito.ma/fr/ahlane/maisons/Maison_...
2,57374711,Marrakech,3500163.7,260.0,Route de Tahanaoute,Villa,4,1,https://www.avito.ma/fr/route_de_tahanaoute/ma...
3,57333583,Casablanca,46002305.4,102.0,Sidi Maarouf,Maison,12,5,https://www.avito.ma/fr/sidi_maarouf/maisons/M...
4,56921942,Casablanca,3500163.7,378.0,Sidi Bernoussi,Maison,4,1,https://www.avito.ma/fr/sidi_bernoussi/maisons...


### Sauvegarde des données nettoyées (Export Final)

In [12]:
# 1. Définition du chemin de sortie
output_dir = "../data/clean_data"
output_path = os.path.join(output_dir, "avito_vendre_clean.csv")

# 2. Création du dossier s'il n'existe pas
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 3. Sauvegarde du DataFrame
df.to_csv(output_path, index=False, encoding='utf-8')

print(f"✅ Félicitations ! Le fichier a été sauvegardé avec succès.")
print(f"📍 Emplacement : {output_path}")
print(f"Nombre de lignes finales : {len(df)}")


✅ Félicitations ! Le fichier a été sauvegardé avec succès.
📍 Emplacement : ../data/clean_data\avito_vendre_clean.csv
Nombre de lignes finales : 5229
